In [1]:
# inspired by VPT cleaning process, use model, preprocess = clip.load('RN50x64', device, jit=True)
#!pip install matplotlib
#!pip install -U yt-dlp
#!pip install ffmpeg-python

import yt_dlp
import matplotlib
from yt_dlp import YoutubeDL
import random
import os


def s_to_HHmmss(secs):
    mins = (secs - secs%60) / 60
    secs = secs%60
    hours = (mins - mins%60) / 60
    mins = mins%60
    
    if hours>0:
        HHmmss = ":".join([str(int(hours)),str(int(mins)),str(secs)])
    elif mins>0:
        HHmmss = ":".join([str(int(mins)),str(secs)])
    else:
        HHmmss = str(secs)
            
    return HHmmss

Duplicate key in file PosixPath('/home/idmi/miniconda3/envs/minerl/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 603 ('image.interpolation:\tnone')


In [2]:
# OPEN VIDEO FILE HEADER, READ SEEN LINKS/CHANNELS
header_file = open("spamham_samples/header.txt", "r")
header_text = header_file.readlines()
header_file.close()

all_links = []
all_channels = []

# https://pypi.org/project/yt-dlp/#usage-and-options
def longer_than_a_minute(info, *, incomplete):
    """Download only videos longer than 10 minutes"""
    duration = info.get('duration')
    if duration and duration < 600:
        return 'The video is too short'
YDL_OPTIONS = {'noplaylist':True, 'Download':False, 'match_filter':longer_than_a_minute, 'ignoreerrors':True}

for line in header_text:
    link = line
    if (not (link in all_links)) and (link!=''):
        all_links.append(link)

In [3]:
search_queries = [
#    "minecraft fresh start new world creative",
#    "minecraft new world modded",
#    "minecraft guide",
#    "minecraft speedrun",
#    "minecraft how to redstone",
#    "minecraft new world tutorial",
#    "minecraft creative guide",
#    "minecraft tutorial how to survival vanilla new world lets play",
#    "minecraft basics",
#    "minecraft for beginners",
#    "minecraft how to hardcore guide",
#    "minecraft how to survival 1.16 guide",
    "minecraft stream",
    "minecraft lets play",
    "minecraft gameplay survival",
    "minecraft gameplay hardcore",
    "minecraft lets play",
    "minecraft tutorial",
    "minecraft new world survival",
    "minecraft hardcore",
    "minecraft gameplay surival",
    "minecraft longplay",
    ]

for query in search_queries:

    search_query = query
    
    #"minecraft tutorial how to hardcore vanilla new world lets play"
    # minecraft vanilla lets play stream
    #"minecraft guide walkthrough survival new world 1.16"
    #"minecraft survival hardcore tutorial how to guide learn basics beginner lets play walkthrough gameplay new world first night first day 1.16"
    #"minecraft new world creative vanilla"
    #"minecraft creative mode how to"
    # "minecraft shaders"
    #"minecraft console edition pocket edition ps3 ps4 ps5 xbox bedrock"
    #"playing minecraft with multiplayer server"
    # minecraft creative 1.16
    # minecraft creative
    # minecraft lets play stream mod
    # minecraft mod technic
    # minecraft mod

    with yt_dlp.YoutubeDL(YDL_OPTIONS) as ydl:
        search_results = ydl.extract_info(f"ytsearchall:{search_query}", download=False)['entries']

    # DOWNLOAD RANDOM FRAMES FROM EACH LINK
    #https://pypi.org/project/yt-dlp/



    # dont re-download frames from videos which have already hadframes downloaded
    with open("spamham_samples/header.txt","r") as used_links:
        used_links = used_links.readlines()
        for i in range(len(used_links)):
            used_links[i] = used_links[i].split('\n')[0]



    # for every link:
    for i in range(len(search_results)):

        
        # separate out video info
        video = search_results[i]
        try:
            video['duration']
        except:
            continue

        if video['duration']>300 and not(video['id'] in used_links):      ############################# PRE-FILTERING FILTERING:

            video_link = video['id']
            video_name = video['title']
            video_channel = video['channel']
            filename = video_link

            print("getting new video!")
            # SELECT RANDOM FRAME
            duration = video['duration']
            random_sec = min(random.randint(0, 10*60), duration-2)
            random_sec_formatted = "*"+s_to_HHmmss(random_sec)+"-"+s_to_HHmmss(random_sec+1)


            # DOWNLOAD SHORT VIDEO SECTION
            full_link = str("https://www.youtube.com/watch?v=%s"%video_link)
            print(full_link)
            print(random_sec_formatted)

            filename_ext = filename+'.mp4'
            command_download_frames = str("yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections %s %s --output spamham_samples/%s.mp4" %(random_sec_formatted, full_link, filename))
            #command_get_1_frame = str('ffmpeg -i spamham_samples/%s -vcodec png -ss 0 -vframes 1 -an -f rawvideo spamham_samples/frames/%s.png -y' %(filename_ext, filename) )
            command_get_1_frame = str('ffmpeg -i spamham_samples/%s -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/%s.jpeg -y' %(filename_ext, filename) )
            command_del_video = str('rm spamham_samples/%s.mp4' %filename)
            command_all_in_1 = command_download_frames +' && ' + command_get_1_frame +' && ' + command_del_video
            os.system(command_all_in_1)

            # CROP VIDEO SECTION TO FIRST FRAME
            #"https://www.youtube.com/watch?v={sID}".format(sID=video_link)
            #label = get_response(filename+".jpg")
            line = str(filename+"\n")

            header_file = open("spamham_samples/header.txt","a")
            sources_file = open("spamham_samples/sources.txt","a")
            header_file.write(line)
            sources_file.write(video_channel+","+"https://www.youtube.com/watch?v={sID}".format(sID=video_link)+"\n")
            header_file.close()
            sources_file.close()

[youtube:search] Extracting URL: ytsearchall:minecraft stream
[download] Downloading playlist: minecraft stream
[youtube:search] query "minecraft stream": Downloading web client config
[youtube:search] query "minecraft stream" page 1: Downloading API JSON
[youtube:search] query "minecraft stream" page 2: Downloading API JSON
[youtube:search] query "minecraft stream" page 3: Downloading API JSON
[youtube:search] query "minecraft stream" page 4: Downloading API JSON
[youtube:search] query "minecraft stream" page 5: Downloading API JSON
[youtube:search] query "minecraft stream" page 6: Downloading API JSON
[youtube:search] query "minecraft stream" page 7: Downloading API JSON
[youtube:search] query "minecraft stream" page 8: Downloading API JSON
[youtube:search] query "minecraft stream" page 9: Downloading API JSON
[youtube:search] query "minecraft stream" page 10: Downloading API JSON
[youtube:search] query "minecraft stream" page 11: Downloading API JSON
[youtube:search] query "minecraf